导入基本包，并配置超参数，创建数据集Loader。

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01


train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

创建RNN网络。

In [2]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN,self).__init__()
        self.hidden_size=hidden_size
        self.num_layers = num_layers
        self.lstm=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self,x):
        # LSTM的状态量
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # 前向传播LSTM,输出(batch_size, seq_length, hidden_size)的tensor
        out, _ = self.lstm(x, (h0, c0))
        
        # 获取LSTM最后一个节点输出用于全连接
        out = self.fc(out[:, -1, :])
        return out
        

创建模型

In [4]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')

214,282 total parameters.


训练

In [6]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # 前向计算
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播与参数优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5516
Epoch [1/2], Step [200/600], Loss: 0.1758
Epoch [1/2], Step [300/600], Loss: 0.3404
Epoch [1/2], Step [400/600], Loss: 0.1379
Epoch [1/2], Step [500/600], Loss: 0.1143
Epoch [1/2], Step [600/600], Loss: 0.1140
Epoch [2/2], Step [100/600], Loss: 0.1665
Epoch [2/2], Step [200/600], Loss: 0.0841
Epoch [2/2], Step [300/600], Loss: 0.0824
Epoch [2/2], Step [400/600], Loss: 0.0765
Epoch [2/2], Step [500/600], Loss: 0.1223
Epoch [2/2], Step [600/600], Loss: 0.1639


测试

In [7]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 97.79 %


保存

In [ ]:
torch.save(model.state_dict(), 'model.ckpt')